In [1]:
import pandas as pd
from pandasql import sqldf
import warnings

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    Transactions_info = pd.read_excel('Transactions_info.xlsx', sheet_name='TRANSACTIONS')
    Customer_info = pd.read_excel('Customer_info.xlsx', sheet_name='QUERY_FOR_ABT_CUSTOMERINFO_0002')

- вывести список клиентов с непрерывной историей за год, средний чек за период, средняя сумма покупок за месяц, количество всех операций по клиенту за период

Уточнение: считал что клиент имеет непрерывную историю за год если параметр Tenure = 12

In [2]:
sqldf("""

SELECT T.ID_client, 
       ROUND(AVG(T.Check_sum_payment), 2) AS Avg_check_for_the_period,
       ROUND(SUM(T.Check_sum_payment)/12, 2) AS Avg_monthly_purchases,
       SUM(Num_operations) AS Number_operations_for_the_period
       
FROM (SELECT Id_check, date_new, ID_client, SUM(Count_products) AS Check_count_products, 
        SUM(Sum_payment) AS Check_sum_payment,
        COUNT(ID_client) AS Num_operations
        FROM Transactions_info
        GROUP BY Id_check, date_new, ID_client
        HAVING ID_client IN (SELECT  C.Id_client FROM Customer_info AS C WHERE Tenure = 12)) AS T
        
GROUP BY T.ID_client

""")

,ID_client,Avg_check_for_the_period,Avg_monthly_purchases,Number_operations_for_the_period
0,33297,101.15,33.72,40
1,114389,65.09,5.42,7
2,114395,108.46,379.61,446
3,131083,82.68,6.89,8
4,166395,117.70,156.93,194
...,...,...,...,...
299,282702,74.56,12.43,22
300,283232,90.75,302.51,431
301,283356,69.73,11.62,13
302,283448,96.21,72.16,124


- вывести помесячную информацию: средняя сумма чека в месяц, среднее количество операций в месяц, среднее количество клиентов, которые совершали операции; долю от общего количества операций за год и долю в месяц от общей суммы операций; вывести % соотношение M/F/NA в каждом месяце с их долей затрат

In [3]:
sqldf("""

SELECT strftime('%m, %Y', date_new) AS Month,
        ROUND(AVG(Check_sum_payment), 2) AS Avg_sum_check_per_month,
        ROUND(AVG(Num_operations), 1) AS Avg_number_of_operations_per_month,      
        COUNT(ID_client) AS Num_of_clients_made_operations,
        ROUND(SUM(Num_operations)*100.0/(SELECT COUNT(ID_client) FROM Transactions_info),2) 
                                                            AS '%num_of_operations_per_year',
        ROUND(SUM(Check_sum_payment)*100.0/(SELECT SUM(Sum_payment) FROM Transactions_info),2) 
                                                            AS '%sum_of_operations_per_year',
        ROUND(SUM(CASE WHEN Gender = 'F' THEN Check_sum_payment ELSE 0 END)*100.0/
                                                                SUM(Check_sum_payment),2) AS '%F',
        ROUND(SUM(CASE WHEN Gender = 'M' THEN Check_sum_payment ELSE 0 END)*100.0/
                                                                SUM(Check_sum_payment),2) AS '%M',
        ROUND(SUM(CASE WHEN Gender is Null THEN Check_sum_payment ELSE 0 END)*100.0/
                                                                SUM(Check_sum_payment),2) AS '%NA'

  
FROM (SELECT Id_check, date_new, T.ID_client, 
        SUM(Sum_payment) AS Check_sum_payment,
        COUNT(T.ID_client) AS Num_operations,
        C.Gender
        FROM Transactions_info AS T
        LEFT JOIN (SELECT Id_client, Gender FROM Customer_info) AS C ON T.ID_client = C.Id_client
        GROUP BY Id_check, T.ID_client, date_new) 
        
GROUP BY date_new
""")

,Month,Avg_sum_check_per_month,Avg_number_of_operations_per_month,Num_of_clients_made_operations,%num_of_operations_per_year,%sum_of_operations_per_year,%F,%M,%NA
0,"06, 2015",95.26,10.2,316,0.77,0.76,72.52,25.31,2.17
1,"07, 2015",93.85,10.1,2929,7.09,6.91,73.27,24.60,2.13
2,"08, 2015",91.48,10.1,2862,6.89,6.58,76.24,21.40,2.36
3,"09, 2015",93.24,10.1,2794,6.70,6.55,74.89,22.40,2.71
4,"10, 2015",94.20,10.1,2936,7.08,6.95,73.27,24.59,2.14
5,"11, 2015",90.30,9.8,2794,6.52,6.34,73.44,24.20,2.36
6,"12, 2015",91.64,9.8,3139,7.36,7.23,70.47,27.07,2.45
7,"01, 2016",90.18,10.0,3052,7.25,6.92,70.78,26.58,2.65
8,"02, 2016",103.07,10.3,4681,11.49,12.13,70.80,26.44,2.76
9,"03, 2016",95.85,9.7,4467,10.32,10.76,72.16,24.83,3.01


- Вывести возрастные группы клиентов с шагом 10 лет и отдельно клиентов, у которых нет данной информации с параметрами сумма и количество операций за весь период, и поквартально, средние показатели и %.

In [4]:
Customer_info['Age_group'] = ''
Transactions_info['Q'] = ''


# sqldf("""

# UPDATE Customer_info SET 
#     Age_group = CASE 
#       WHEN Age BETWEEN 1 AND 10 THEN '1-10' 
#       WHEN Age BETWEEN 11 AND 20 THEN '11-20'
#       WHEN Age BETWEEN 21 AND 30 THEN '21-30'
#       WHEN Age BETWEEN 31 AND 40 THEN '31-40'
#       WHEN Age BETWEEN 41 AND 50 THEN '41-50'
#       WHEN Age BETWEEN 51 AND 60 THEN '51-60'
#       WHEN Age BETWEEN 61 AND 70 THEN '61-70'
#       WHEN Age BETWEEN 71 AND 80 THEN '71-80'
#       WHEN Age BETWEEN 81 AND 88 THEN '81-88'
#         ELSE 'NaN' 
#       END;

# UPDATE Transactions_info SET 
#     Q = CASE 
#       WHEN date_new < '2015-07-01' THEN 'Q2.2015' 
#       WHEN date_new BETWEEN '2015-07-01' AND '2015-10-01' THEN 'Q3.2015'
#       WHEN date_new BETWEEN '2015-10-01' AND '2016-01-01' THEN 'Q4.2015'
#       WHEN date_new BETWEEN '2016-01-01' AND '2016-04-01' THEN 'Q1.2016'
#       WHEN date_new BETWEEN '2016-04-01' AND '2016-07-01' THEN 'Q2.2016'
#       END;

# """)

Customer_info.loc[Customer_info['Age'] < 11, 'Age_group'] = '1-10'
Customer_info.loc[(Customer_info['Age'] >= 11)&(Customer_info['Age'] < 21), 'Age_group'] = '11-20'
Customer_info.loc[(Customer_info['Age'] >= 21)&(Customer_info['Age'] < 31), 'Age_group'] = '21-30'
Customer_info.loc[(Customer_info['Age'] >= 31)&(Customer_info['Age'] < 41), 'Age_group'] = '31-40'
Customer_info.loc[(Customer_info['Age'] >= 41)&(Customer_info['Age'] < 51), 'Age_group'] = '41-50'
Customer_info.loc[(Customer_info['Age'] >= 51)&(Customer_info['Age'] < 61), 'Age_group'] = '51-60'
Customer_info.loc[(Customer_info['Age'] >= 61)&(Customer_info['Age'] < 71), 'Age_group'] = '61-70'
Customer_info.loc[(Customer_info['Age'] >= 71)&(Customer_info['Age'] < 81), 'Age_group'] = '71-80'
Customer_info.loc[(Customer_info['Age'] >= 81)&(Customer_info['Age'] < 90), 'Age_group'] = '81-88'

Transactions_info.loc[Transactions_info['date_new'] < '2015-07-01', 'Q'] = 'Q2.2015'
Transactions_info.loc[(Transactions_info['date_new'] < '2015-10-01')& 
                      (Transactions_info['date_new'] >= '2015-07-01'), 'Q'] = 'Q3.2015'
Transactions_info.loc[(Transactions_info['date_new'] < '2016-01-01')& 
                      (Transactions_info['date_new'] >= '2015-10-01'), 'Q'] = 'Q4.2015'
Transactions_info.loc[(Transactions_info['date_new'] < '2016-04-01')& 
                      (Transactions_info['date_new'] >= '2016-01-01'), 'Q'] = 'Q1.2016'
Transactions_info.loc[(Transactions_info['date_new'] < '2016-07-01')& 
                      (Transactions_info['date_new'] >= '2016-04-01'), 'Q'] = 'Q2.2016'

In [5]:
sqldf("""

SELECT DISTINCT Age_group, 
 Sum_Q2_2015,
 Sum_Q3_2015,
 Sum_Q4_2015,
 Sum_Q1_2016,
 Sum_Q2_2016,
 SUM(Sum_payment) OVER(PARTITION BY Age_group) AS Sum_operations_for_period,
 ROUND((Sum_Q2_2015+Sum_Q3_2015+Sum_Q4_2015+Sum_Q1_2016+Sum_Q2_2016)*100/
                (SELECT SUM(Sum_payment) FROM Transactions_info), 2) AS '%_of_Total_sum',
 ROUND((Sum_Q3_2015+Sum_Q4_2015+Sum_Q1_2016+Sum_Q2_2016)/4, 2) AS 'Mean_sum_Q3_2015-Q2_2016',
 Count_Q2_2015,
 Count_Q3_2015,
 Count_Q4_2015,
 Count_Q1_2016,
 Count_Q2_2016,
 COUNT(Sum_payment) OVER(PARTITION BY Age_group) AS Count_operations_for_period,
 ROUND((Count_Q2_2015+Count_Q3_2015+Count_Q4_2015+Count_Q1_2016+Count_Q2_2016)*100.0/
                (SELECT COUNT(Sum_payment) FROM Transactions_info), 2) AS '%_of_Total_count',
 (Count_Q3_2015+Count_Q4_2015+Count_Q1_2016+Count_Q2_2016)/4 AS 'Mean_count_Q3_2015-Q2_2016'
FROM Transactions_info AS T
LEFT JOIN (SELECT Id_client, Age_group FROM Customer_info) AS C ON T.ID_client = C.Id_client
LEFT JOIN (SELECT Age_group AS Age_group_Q2_2015, SUM(Sum_payment) AS Sum_Q2_2015, 
            COUNT(Sum_payment) AS Count_Q2_2015
            FROM Transactions_info AS T_Q2_2015
            LEFT JOIN (SELECT Id_client, Age_group FROM Customer_info) AS C ON T_Q2_2015.ID_client = C.Id_client
            GROUP BY Q, Age_group
            HAVING Q = 'Q2.2015') ON Age_group_Q2_2015 = C.Age_group
LEFT JOIN (SELECT Age_group AS Age_group_Q3_2015, SUM(Sum_payment) AS Sum_Q3_2015,
            COUNT(Sum_payment) AS Count_Q3_2015
            FROM Transactions_info AS T_Q3_2015
            LEFT JOIN (SELECT Id_client, Age_group FROM Customer_info) AS C ON T_Q3_2015.ID_client = C.Id_client
            GROUP BY Q, Age_group
            HAVING Q = 'Q3.2015') ON Age_group_Q3_2015 = C.Age_group
LEFT JOIN (SELECT Age_group AS Age_group_Q4_2015, SUM(Sum_payment) AS Sum_Q4_2015,
            COUNT(Sum_payment) AS Count_Q4_2015
            FROM Transactions_info AS T_Q4_2015
            LEFT JOIN (SELECT Id_client, Age_group FROM Customer_info) AS C ON T_Q4_2015.ID_client = C.Id_client
            GROUP BY Q, Age_group
            HAVING Q = 'Q4.2015') ON Age_group_Q4_2015 = C.Age_group
LEFT JOIN (SELECT Age_group AS Age_group_Q1_2016, SUM(Sum_payment) AS Sum_Q1_2016,
            COUNT(Sum_payment) AS Count_Q1_2016
            FROM Transactions_info AS T_Q1_2016
            LEFT JOIN (SELECT Id_client, Age_group FROM Customer_info) AS C ON T_Q1_2016.ID_client = C.Id_client
            GROUP BY Q, Age_group
            HAVING Q = 'Q1.2016') ON Age_group_Q1_2016 = C.Age_group
LEFT JOIN (SELECT Age_group AS Age_group_Q2_2016, SUM(Sum_payment) AS Sum_Q2_2016,
            COUNT(Sum_payment) AS Count_Q2_2016
            FROM Transactions_info AS T_Q2_2016
            LEFT JOIN (SELECT Id_client, Age_group FROM Customer_info) AS C ON T_Q2_2016.ID_client = C.Id_client
            GROUP BY Q, Age_group
            HAVING Q = 'Q2.2016') ON Age_group_Q2_2016 = C.Age_group
            
 
""")

,Age_group,Sum_Q2_2015,Sum_Q3_2015,Sum_Q4_2015,Sum_Q1_2016,Sum_Q2_2016,Sum_operations_for_period,%_of_Total_sum,Mean_sum_Q3_2015-Q2_2016,Count_Q2_2015,Count_Q3_2015,Count_Q4_2015,Count_Q1_2016,Count_Q2_2016,Count_operations_for_period,%_of_Total_count,Mean_count_Q3_2015-Q2_2016
0,,409.78,11477.04,11448.64,20265.34,21040.44,64641.24,1.63,16057.86,40,1239,1225,1953,2092,6549,1.56,1627
1,1-10,88.11,1973.71,3436.17,4471.20,2078.62,12047.81,0.30,2989.93,8,203,360,403,190,1164,0.28,289
2,11-20,949.12,31557.94,42155.86,54165.20,50151.31,178979.43,4.50,44507.58,106,3654,4526,5339,5286,18911,4.51,4701
3,21-30,5955.00,151995.49,173711.35,246322.91,262311.41,840296.16,21.12,208585.29,666,16844,18815,24656,26101,87082,20.78,21604
4,31-40,5730.81,152710.67,155681.33,228806.47,216654.02,759583.30,19.10,188463.12,625,16432,16738,24044,22775,80614,19.23,19997
5,41-50,4784.33,121614.33,109827.09,160517.11,154713.20,551456.06,13.86,136667.93,456,11809,11330,15560,15344,54499,13.00,13510
6,51-60,2880.24,99072.93,108459.17,161326.87,153442.94,525182.15,13.20,130575.48,315,10805,11835,16146,16255,55356,13.21,13760
7,61-70,7767.19,197482.17,182683.28,273848.68,244462.03,906243.35,22.78,224619.04,849,22371,19849,30110,27450,100629,24.01,24945
8,71-80,1435.76,28266.13,27094.41,33845.72,41897.14,132539.16,3.33,32775.85,142,3191,2978,3385,3925,13621,3.25,3369
9,81-88,100.82,1073.05,2025.16,2274.26,1291.66,6764.95,0.17,1666.03,12,128,204,229,124,697,0.17,171
